In [2]:
import cv2
import numpy as np
import easyocr

# Função para encontrar o contorno da placa
def encontrar_contorno_placa(img):
    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplicar um filtro para suavizar a imagem e reduzir ruídos
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detectar bordas usando o algoritmo de Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Encontrar contornos na imagem
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Inicializar variáveis para armazenar o maior retângulo encontrado
    placa_contorno = None
    max_area = 0

    # Iterar sobre todos os contornos encontrados
    for contour in contours:
        # Aproximar o contorno para verificar se é um retângulo
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar se o contorno tem 4 vértices (é um retângulo)
        if len(approx) == 4:
            # Calcular a área do contorno
            area = cv2.contourArea(approx)

            # Considerar o contorno como a placa se for o maior retângulo encontrado
            if area > max_area:
                max_area = area
                placa_contorno = approx

    # Se um contorno da placa for encontrado
    if placa_contorno is not None:
        # Desenhar o contorno na imagem original para visualização
        cv2.drawContours(img, [placa_contorno], -1, (0, 255, 0), 3)

        # Extrair os pontos do contorno
        pts = placa_contorno.reshape(4, 2)

        # Ordenar os pontos para obter o retângulo corretamente
        rect = np.zeros((4, 2), dtype="float32")
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # Top-left
        rect[2] = pts[np.argmax(s)]  # Bottom-right

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # Top-right
        rect[3] = pts[np.argmax(diff)]  # Bottom-left

        # Definir a largura e altura do novo retângulo alinhado
        (tl, tr, br, bl) = rect
        widthA = np.linalg.norm(br - bl)
        widthB = np.linalg.norm(tr - tl)
        maxWidth = max(int(widthA), int(widthB))

        heightA = np.linalg.norm(tr - br)
        heightB = np.linalg.norm(tl - bl)
        maxHeight = max(int(heightA), int(heightB))

        # Definir os pontos do destino para "retificar" a placa
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")

        # Calcular a transformação de perspectiva
        M = cv2.getPerspectiveTransform(rect, dst)

        # Aplicar a transformação de perspectiva para "retificar" a placa
        warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))

        return warped, img  # Retornar a imagem retificada e a imagem com o contorno desenhado
    else:
        return None, img  # Nenhum contorno da placa encontrado

# Função para realizar OCR na imagem da placa
def detectar_caracteres(img):
    # Inicializar o leitor do EasyOCR
    leitor = easyocr.Reader(['pt', 'en'])  # Definimos os idiomas como português e inglês

    # Definir a lista de caracteres permitidos (apenas letras maiúsculas e números)
    allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

    # Fazer OCR na imagem
    resultado = leitor.readtext(img, allowlist=allowlist)

    # Extrair o texto detectado
    texto_detectado = " ".join([res[1] for res in resultado])
    return texto_detectado

# Função para processar o vídeo frame a frame e salvar o resultado
def processar_video(video_path, output_path):
    # Carregar o vídeo
    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi carregado com sucesso
    if not cap.isOpened():
        print("Erro ao carregar o vídeo.")
        return

    # Definir o codec e criar o objeto VideoWriter para salvar o vídeo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Obter FPS do vídeo original
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Largura do vídeo
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Altura do vídeo
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Loop para processar o vídeo frame a frame
    while cap.isOpened():
        ret, frame = cap.read()

        # Se o frame foi lido corretamente
        if ret:
            # Encontrar o contorno da placa
            placa_alinhada, frame_com_contorno = encontrar_contorno_placa(frame)

            # Se a placa foi encontrada, realizar OCR
            if placa_alinhada is not None:
                # Detectar caracteres na placa
                texto_detectado = detectar_caracteres(placa_alinhada)

                # Exibir o texto detectado no frame
                cv2.putText(frame_com_contorno, f"Placa: {texto_detectado}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # ** Imprimir o texto detectado **
                print(f"Placa detectada: {texto_detectado}")

            # Gravar o frame no arquivo de saída
            out.write(frame_com_contorno)

        else:
            break

    # Liberar o vídeo e o arquivo de saída
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Chamar a função com o caminho do vídeo de entrada e de saída
video_path = 'organic-data/video_teste_2.mp4'
output_path = 'resultado_processado.mp4'
processar_video(video_path, output_path)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Placa detectada: 


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Placa detectada: 


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Placa detectada: OFEI9


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Placa detectada: OFEI9
Placa detectada: OFEI9


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
